# Preparación del Entorno

- La solución inicial, a pesar de aparentemente estar bien el resumen de cantidad de clases, parece no estar bien la primer solución en el Zulip (según https://dmeyf2023.zulip.rebelare.com/#narrow/stream/412-CazaTalentos/topic/Enunciado/near/48562, el kaggle público debería ser 40.27)
- Voy a probar un código basado en este posteo <https://dmeyf2023.zulip.rebelare.com/#narrow/stream/401-Code/topic/calculo.20de.20clase_ternaria.20en.20R.20con.20TV.20y.20data.2Etable/near/48079> a ver si da bien , y luego lo voy revisando

## Importación de Librerías

In [1]:
library(data.table)

In [2]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("rpart.plot")

Loading required package: rpart

Loading required package: rpart.plot



## Importación del archivo

In [3]:
# Aqui se debe poner la carpeta de la materia de SU computadora local
setwd("C:/Users/German/Desktop/dmeyf2023/German/Clase 1") # Establezco el Working Directory

# Resolución del problema

## Etiquetado del target

In [4]:
dt <- data.table::fread(file="competencia_01_crudo.csv", header = T)

str(dt)

#1.Cálculo de la variable "clase_ternaria" como el valor máimo de foto_mes del cliente menos el valor minimo (ej, 3 meses = 2; 2 meses = 1; 1 mes = 0)
#2.De ahí, sólo los que tienen el menor valor de foto_mes y al valor de clase ternaria se reemplaza con "CONTINUA". Si es igual a la diferencia máxima entre los valores de foto_mes, o en caso contrario con el valor "BAJA+" +  la direrencia + 1

target <- dt[, list(foto_mes, clase_ternaria = (max(foto_mes) - min(foto_mes))), by = c("numero_de_cliente")][foto_mes == min(foto_mes), c(list(numero_de_cliente = numero_de_cliente),
                                                                                                                                           lapply(.SD, function(x){
                                                                                                                                             ifelse(x == (max(dt[, unique(foto_mes)]) - min(dt[, unique(foto_mes)])), "Continua", paste0("BAJA+", (x+1)))
                                                                                                                                           })),
                                                                                                              .SDcols = c("clase_ternaria")]

competencia_01 <- merge(dt, target, by = "numero_de_cliente", all.x = TRUE)


Classes 'data.table' and 'data.frame':	491063 obs. of  154 variables:
 $ numero_de_cliente                   : int  29183981 29184630 29185433 29185587 29185646 29187499 29188277 29188490 29188883 29189950 ...
 $ foto_mes                            : int  202103 202103 202103 202103 202103 202103 202103 202103 202103 202103 ...
 $ active_quarter                      : int  1 1 1 1 1 1 1 1 1 1 ...
 $ cliente_vip                         : int  0 0 0 0 0 0 0 0 0 0 ...
 $ internet                            : int  0 0 0 0 0 0 0 0 0 0 ...
 $ cliente_edad                        : int  50 59 68 79 60 71 67 45 66 59 ...
 $ cliente_antiguedad                  : int  197 322 268 322 257 375 322 307 322 266 ...
 $ mrentabilidad                       : num  14469 11902 847 4977 2860 ...
 $ mrentabilidad_annual                : num  125765 74159 21672 47736 37801 ...
 $ mcomisiones                         : num  2390 18751 482 1839 4035 ...
 $ mactivos_margen                     : num  7751 -8624 -

In [5]:
# Contar la cantidad de distintos campos agrupados en la columna "clase_ternaria"
count_clase <- competencia_01[, .N, by = clase_ternaria]
count_clase

clase_ternaria,N
<chr>,<int>
Continua,483935
NA,4153
BAJA+1,1049
BAJA+2,1926


In [6]:
#Guardar archivo en la carpeta
data.table::fwrite(competencia_01, file="competencia_01_alternativo.csv")

In [7]:
#Extra, para el Zulip: cuántos sólo en 202103

# Filtrar el DataFrame para obtener solo los registros con fecha_mes igual a 202103
subset_dataset <- competencia_01[foto_mes %in% 202103]

# Contar en este subset
count_clase <- subset_dataset[, .N, by = clase_ternaria]
count_clase

clase_ternaria,N
<chr>,<int>
Continua,161312
BAJA+1,1049
BAJA+2,963
